In [ ]:
from huggingface_hub import login
login("")

# Load Data

In [1]:
import pandas as pd
base = 'dataset_generation/data/'
files = ['bbc.txt', 'cnn.txt', 'fox.txt', 'ndtv.txt', 'nyt.txt', 'wsj.txt']
df = pd.DataFrame()
# df = pd.read_csv(base+files[0])
# source = files[0][:files[0].index('.')]
for file in files:
    source = file[:file.index('.')]
    print(source)
    temp = pd.read_csv(base+file)
    temp['source'] = source
    if source == 'fox':
        temp = temp.sample(frac=.6)
    df = df.append(temp)
print(len(df))
df.drop('Unnamed: 0', axis=1, inplace=True)
df.loc[df.is_article == 2, 'is_article'] = 1
df.drop('source', axis=1, inplace=True)
df.rename(columns={'is_article': 'label'}, inplace=True)
df.head()


bbc
cnn
fox
ndtv
nyt
wsj
2431


/tmp/ipykernel_14998/60339003.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp)
/tmp/ipykernel_14998/60339003.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp)
/tmp/ipykernel_14998/60339003.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp)
/tmp/ipykernel_14998/60339003.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp)
/tmp/ipykernel_14998/60339003.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp)
/tmp/ipykernel_14998/60339003.py:14

,html,label
0,"<a class=""sc-4befc967-1 gzOvMy"" data-testid=""i...",0
1,"<section class=""sc-ba4a1d3a-0 fXIgdM""><nav cla...",0
2,"<a class=""sc-ba4a1d3a-4 hIyPug"" data-testid=""i...",0
3,"<a class=""sc-ba4a1d3a-4 zsFYm"" data-testid=""in...",0
4,"<a class=""sc-ba4a1d3a-4 hIyPug"" data-testid=""i...",0


In [2]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, shuffle=True, stratify=df['label'])
print("train_df")
print("Total: ", len(train_df))
print("% article: ", len(train_df[train_df.label > 0])/len(train_df))
print()
print("test_df")
print("Total: ", len(test_df))
print("% article: ", len(test_df[test_df.label > 0])/len(test_df))

train_df
Total:  1823
% article:  0.43115743280307184

test_df
Total:  608
% article:  0.4309210526315789


In [3]:
import datasets
from datasets import Dataset, DatasetDict

train = Dataset.from_pandas(train_df)
test = Dataset.from_pandas(test_df)
train = train.remove_columns('__index_level_0__')
test = test.remove_columns('__index_level_0__')

dataset = DatasetDict()
dataset['train'] = train
dataset['test'] = test

/home/mananj2/miniconda3/envs/wc/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Model

In [4]:
from transformers import AutoTokenizer

model_name = 'models/classifier_model/'
tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

0

In [5]:
def preprocess_function(examples):
    return tokenizer(examples["html"], truncation=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1823 [00:00<?, ? examples/s]

Map: 100%|██████████| 608/608 [00:00<00:00, 2496.05 examples/s]


In [6]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

2024-01-11 18:21:35.663371: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-11 18:21:35.663445: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-11 18:21:35.664903: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-11 18:21:35.673657: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-11 18:21:38.477390: W tensorflow/comp

In [7]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [8]:
import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, 
    # device_map='auto',
    # torch_dtype=torch.float16,
    # use_auth_token=True,
    # load_in_8bit=True
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
training_args = TrainingArguments(
    output_dir="models",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [11]:
trainer.train()

/home/mananj2/miniconda3/envs/wc/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/mananj2/miniconda3/envs/wc/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.411700,0.241507,0.944079,0.943452,0.941391,0.947624
2,0.198800,0.127485,0.972039,0.971631,0.969881,0.974044
3,0.136300,0.094701,0.988487,0.988279,0.987651,0.988958
4,0.046700,0.093270,0.988487,0.988279,0.987651,0.988958
5,0.007200,0.095773,0.986842,0.986598,0.986170,0.987049


/home/mananj2/miniconda3/envs/wc/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/mananj2/miniconda3/envs/wc/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/mananj2/miniconda3/envs/wc/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/mananj2/miniconda3/envs/wc/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was a

TrainOutput(global_step=3040, training_loss=0.1343232150508189, metrics={'train_runtime': 696.8289, 'train_samples_per_second': 13.081, 'train_steps_per_second': 4.363, 'total_flos': 1575621541138500.0, 'train_loss': 0.1343232150508189, 'epoch': 5.0})

In [11]:
trainer.evaluate()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


/home/mananj2/miniconda3/envs/wc/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.02863984927535057,
 'eval_accuracy': 0.9950657894736842,
 'eval_f1': 0.9949720901385156,
 'eval_precision': 0.9947484432688599,
 'eval_recall': 0.9952014296430305,
 'eval_runtime': 20.2036,
 'eval_samples_per_second': 30.094,
 'eval_steps_per_second': 10.048}